<a href="https://colab.research.google.com/github/leochame/Computer-Study-Note/blob/main/llm/llms-from-srcatch/ch04.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#
注意力机制是LLM的核心，那么接下来我们将组装剩下的组件

In [2]:
from importlib.metadata import version

import matplotlib
import tiktoken
import torch

print("matplotlib version:", version("matplotlib"))
print("torch version:", version("torch"))
print("tiktoken version:", version("tiktoken"))
#加载并确认版本

matplotlib version: 3.10.0
torch version: 2.6.0+cu124
tiktoken version: 0.9.0


In [3]:
GPT_CONFIG_124M = {
    "vocab_size": 50257,    # Vocabulary size
    "context_length": 1024, # Context length
    "emb_dim": 768,         # Embedding dimension
    "n_heads": 12,          # Number of attention heads
    "n_layers": 12,         # Number of layers
    "drop_rate": 0.1,       # Dropout rate
    "qkv_bias": False       # Query-Key-Value bias
}
#初始化定义需要的各种超参数

上面是配置，

<img src="https://sebastianraschka.com/images/LLMs-from-scratch-images/ch04_compressed/03.webp" width="500px">

我们编写GPT架构的步骤是：首先从GPT主干入手，创建一个占位符架构；然后实现各个核心组件；最后将它们组装成Transformer块，形成完整的GPT架构。

上方图中的编号框展示了我们处理编写最终GPT架构所需的各个概念的顺序。我们将从第(1)步开始，创建一个名为DummyGPTModel的占位符GPT主干部分

In [4]:
import torch
import torch.nn as nn


class DummyGPTModel(nn.Module):
    def __init__(self, cfg):
        super().__init__()
        self.tok_emb = nn.Embedding(cfg["vocab_size"], cfg["emb_dim"])
        # 词嵌入层，将输入索引转换为词向量，词表大小由字典大小和特征维度决定。
        self.pos_emb = nn.Embedding(cfg["context_length"], cfg["emb_dim"])
        # 位置信息嵌入层，基于文本长度和特征维度生成位置信息。
        self.drop_emb = nn.Dropout(cfg["drop_rate"])
        # Dropout 层，用于随机丢弃一部分嵌入信息以减少过拟合。

        ### 使用多个 Transformer 块（占位符）
        self.trf_blocks = nn.Sequential(
            *[DummyTransformerBlock(cfg) for _ in range(cfg["n_layers"])]
        )
        # Transformer 模块的堆叠，模型核心部分。

        ### 使用归一化层（占位符）
        self.final_norm = DummyLayerNorm(cfg["emb_dim"])
        # 最终归一化层，用于调整特征分布。

        self.out_head = nn.Linear(
            cfg["emb_dim"], cfg["vocab_size"], bias=False
        )
        # 输出层，将特征映射到词表分布，最终预测输出单词。

    def forward(self, in_idx):
        batch_size, seq_len = in_idx.shape
        # 获取批次大小和序列长度。

        tok_embeds = self.tok_emb(in_idx)
        # 根据输入索引生成词嵌入。
        pos_embeds = self.pos_emb(torch.arange(seq_len, device=in_idx.device))
        # 生成对应的位置信息嵌入。

        x = tok_embeds + pos_embeds
        # 将词嵌入和位置信息嵌入相加。
        x = self.drop_emb(x)
        # 应用 Dropout 随机丢弃部分信息。
        x = self.trf_blocks(x)
        # 通过多个 Transformer 块处理特征。
        x = self.final_norm(x)
        # 应用最终的归一化层。
        logits = self.out_head(x)
        # 将隐藏状态映射到词表分布，生成预测结果。
        return logits


class DummyTransformerBlock(nn.Module):
    # Transformer 块的占位类。
    def __init__(self, cfg):
        super().__init__()
        # 占位，实际模型应实现注意力机制和前馈网络。

    def forward(self, x):
        # 此块不执行任何操作，仅返回输入。
        return x


class DummyLayerNorm(nn.Module):
    # 归一化层的占位类。
    def __init__(self, normalized_shape, eps=1e-5):
        super().__init__()
        # 参数用于模拟 LayerNorm 的接口。

    def forward(self, x):
        # 此层不执行任何操作，仅返回输入。
        return x

上面的类是一个简化版的类 GPT 模型。包括词元和位置潜入，dropout、一系列 Transformer 块，最终层归一化。


forward方法描述了数据在模型中的处理流程：它首先计算输入索引的词元和位置嵌入，然后应用dropout，接着通过Transformer块处理数据，再应用归一化，最后使用线性输出层生成logits。

我们接下来介绍一下 GPT 架构的输入和输出，现在我们先宏观概述数据在GPT模型中的流入和流出过程：

<img src="https://sebastianraschka.com/images/LLMs-from-scratch-images/ch04_compressed/04.webp?123" width="500px">

In [5]:
import tiktoken
import torch

tokenizer = tiktoken.get_encoding("gpt2")

batch = []

txt1 = "Every effort moves you"
txt2 = "Every day holds a"

batch.append(torch.tensor(tokenizer.encode(txt1)))
batch.append(torch.tensor(tokenizer.encode(txt2)))
batch = torch.stack(batch, dim=0)
print(batch)

tensor([[6109, 3626, 6100,  345],
        [6109, 1110, 6622,  257]])


我们看到输出张量包含两行，对应于两个文本样本。每个文本样本由 4 个词元组成。每个词元是一个 50257 dimensions 的向量。

In [6]:
torch.manual_seed(123)
model = DummyGPTModel(GPT_CONFIG_124M)

logits = model(batch)
print("Output shape:", logits.shape)
print(logits)

Output shape: torch.Size([2, 4, 50257])
tensor([[[-0.9289,  0.2748, -0.7557,  ..., -1.6070,  0.2702, -0.5888],
         [-0.4476,  0.1726,  0.5354,  ..., -0.3932,  1.5285,  0.8557],
         [ 0.5680,  1.6053, -0.2155,  ...,  1.1624,  0.1380,  0.7425],
         [ 0.0447,  2.4787, -0.8843,  ...,  1.3219, -0.0864, -0.5856]],

        [[-1.5474, -0.0542, -1.0571,  ..., -1.8061, -0.4494, -0.6747],
         [-0.8422,  0.8243, -0.1098,  ..., -0.1434,  0.2079,  1.2046],
         [ 0.1355,  1.1858, -0.1453,  ...,  0.0869, -0.1590,  0.1552],
         [ 0.1666, -0.8138,  0.2307,  ...,  2.5035, -0.3055, -0.3083]]],
       grad_fn=<UnsafeViewBackward0>)


# 4.2 使用层归一化进行归一化激活

现在我们将实现层归一化。层归一化的主要思想是调整神经网络层的激活（输出）​，使其均值为0且方差（单位方差）为1。
这种方法能够稳定训练过程，并加速权重的高效收敛。
在 Transformer 块中，层归一化会在多头注意力模块的前后应用（我们将在后续实现），并在最终输出层之前再次应用。

我们实现了一个具有5个输入和6个输出的神经网络层，并将其应用于两个输入示例：

In [7]:
torch.manual_seed(123)

# create 2 training examples with 5 dimensions (features) each
batch_example = torch.randn(2, 5)

layer = nn.Sequential(nn.Linear(5, 6), nn.ReLU())
out = layer(batch_example)
print(out)

tensor([[0.2260, 0.3470, 0.0000, 0.2216, 0.0000, 0.0000],
        [0.2133, 0.2394, 0.0000, 0.5198, 0.3297, 0.0000]],
       grad_fn=<ReluBackward0>)


接下来我们执行归一化操作

In [12]:
mean = out.mean(dim=-1, keepdim=True)
var = out.var(dim=-1, keepdim=True)
print("Mean:\n", mean)
print("Variance:\n", var)

Mean:
 tensor([[0.1324],
        [0.2170]], grad_fn=<MeanBackward1>)
Variance:
 tensor([[0.0231],
        [0.0398]], grad_fn=<VarBackward0>)


In [15]:
out_norm = (out - mean) / torch.sqrt(var)
#执行归一化操作
print("Normalized layer outputs:\n", out_norm)

mean = out_norm.mean(dim=-1, keepdim=True)
var = out_norm.var(dim=-1, keepdim=True)

# 关闭科学计数法 去打印张量值
torch.set_printoptions(sci_mode=False)
print("Mean:\n", mean)
print("Variance:\n", var)

Normalized layer outputs:
 tensor([[ 0.6159,  1.4126, -0.8719,  0.5872, -0.8719, -0.8719],
        [-0.0189,  0.1121, -1.0876,  1.5173,  0.5647, -1.0876]],
       grad_fn=<DivBackward0>)
Mean:
 tensor([[    -0.0000],
        [    -0.0000]], grad_fn=<MeanBackward1>)
Variance:
 tensor([[1.0000],
        [1.0000]], grad_fn=<VarBackward0>)


我们将这个过程封装成一个PyTorch模块，以便后续使用

In [17]:
import torch
import torch.nn as nn

# 定义一个神经网络层，专门用于层归一化。
# 继承 PyTorch 的基础类 nn.Module 是标准写法，能让这个类自动管理所有可学习的参数。
class LayerNorm(nn.Module):
    """
    层归一化 (Layer Normalization) 的代码实现。
    """

    # 类的初始化函数。在创建这个层的实例时，会运行一次，用于定义层内部需要的变量和参数。
    # emb_dim: 传入的特征向量维度。例如，如果每个词向量是768维，emb_dim就是768。
    def __init__(self, emb_dim):
        # 调用父类的初始化函数，必须执行。
        super().__init__()

        # 定义一个很小的数值 epsilon。
        # 它的唯一作用是防止在后续计算中，因方差为零而出现除以零的程序错误。
        self.eps = 1e-5

        # 定义一个名为 'scale' 的可学习参数。
        # nn.Parameter() 会将这个张量注册为模型的参数，意味着它的值会在训练过程中被自动学习和更新。
        # 它的功能是对归一化后的数据进行缩放。初始值为1，表示不做任何缩放。
        self.scale = nn.Parameter(torch.ones(emb_dim))

        # 定义一个名为 'shift' 的可学习参数。
        # 同样，它也是一个模型在训练中会自动学习和更新的参数。
        # 它的功能是对归一化后的数据进行平移。初始值为0，表示不做任何平移。
        self.shift = nn.Parameter(torch.zeros(emb_dim))

    # 定义层的前向传播函数。当数据输入到这个层时，这里的代码会被执行。
    # x: 输入的数据张量，形状通常是 [批次大小, 序列长度, 特征维度]。
    def forward(self, x):
        # --- 步骤 1: 计算均值 ---
        # `dim=-1` 指示沿着最后一个维度（即特征维度）计算。
        # `keepdim=True` 保持结果的维度结构，方便后续的向量化计算。
        # 这一步的目的是得到每个数据样本在特征维度上的平均值。
        mean = x.mean(dim=-1, keepdim=True)

        # --- 步骤 2: 计算方差 ---
        # 同样，沿着最后一个维度计算方差。
        # 这一步的目的是得到每个数据样本在特征维度上的数值分散程度。
        # `unbiased=False` 表示使用总体方差进行计算。
        var = x.var(dim=-1, keepdim=True, unbiased=False)

        # --- 步骤 3: 标准化数据 ---
        # 将每个数据点减去其所在样本的均值，再除以其标准差（方差的平方根）。
        # `var + self.eps` 就是为了防止除零错误。
        # 经过这步操作后，输出 `norm_x` 的均值会变为0，方差变为1。
        norm_x = (x - mean) / torch.sqrt(var + self.eps)

        # --- 步骤 4: 应用学习到的缩放和平移 ---
        # 将上一步标准化后的数据 `norm_x`，乘以模型学习到的 `scale` 参数，再加上 `shift` 参数。
        # 这一步给了模型一个“机会”，可以根据任务的需要，将数据调整到它认为最理想的分布状态，
        # 而不只是强制固定在均值为0、方差为1。
        return self.scale * norm_x + self.shift

- 除了通过减去均值并除以方差来执行归一化操作外，我们还引入了两个可训练参数：`scale`（缩放参数）和 `shift`（平移参数）。
- 初始时，`scale` 值为 1，`shift` 值为 0，不会对结果产生影响；但在训练过程中，LLM 会自动调整这两个参数，以提升模型在任务中的表现。
- 这种设计使模型能够学习到最适合其数据的缩放和平移方式。
- 此外，在计算方差的平方根时，我们会添加一个较小的值（`eps`），以避免方差为 0 时出现除以 0 的错误。

In [18]:
ln = LayerNorm(emb_dim=5)
out_ln = ln(batch_example)
mean = out_ln.mean(dim=-1, keepdim=True)
var = out_ln.var(dim=-1, unbiased=False, keepdim=True)
print("Mean:\n", mean)
print("Variance:\n", var)

Mean:
 tensor([[    -0.0000],
        [     0.0000]], grad_fn=<MeanBackward1>)
Variance:
 tensor([[1.0000],
        [1.0000]], grad_fn=<VarBackward0>)
